### 1. 센텐스피스(SentencePiece)

* 논문 : https://arxiv.org/pdf/1808.06226.pdf
* 센텐스피스 깃허브 : https://github.com/google/sentencepiece


----
* 기존 BPE에서는 서브단어를 분리하기 위해 먼저 단어 토큰화가 진행되어야 하나, 한국어와 같은 언어는 토큰화부터 쉽지 않음
* 이와 같이 사전 토큰화 작업(pretokenization) 없이 전처리하지 않은 데이터에 바로 단어 분리 토크나이저를 사용할 수 있는 것이 센텐스피스 알고리즘임

In [1]:
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.1/1.1 MB 14.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2. IMDB 리뷰 토큰화하기

In [2]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [3]:
## 1. 데이터 로드와 데이터프레임으로 저장
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", \
                           filename="IMDB_Reviews.csv")

train_df = pd.read_csv("IMDB_Reviews.csv")
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [4]:
# 2. 5만개 샘플을 센텐스피스 입력으로 사용하기 위해 데이터 프레임을 txt파일로 저장
with open("imdb_review.txt", 'w', encoding='utf-8') as f:
    f.write('\n'.join(train_df['review']))

In [6]:
# 3, 센텐스피스로 단어집합과 각 단어에 고유한 정수 부여
spm.SentencePieceTrainer.Train(
    '--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

* input : 학습시킬 파일명
* model_prefix : 만들어질 모델명
* vocab_size : 단어집합 크기
* model_type : 사용할 알고리즘(unigram이 디폴트. bpe, char, word 등이 있음)
* max_sentence_length : 최대 문장길이
* pad_id, pad_piece: pad token id, 값
* unk_id, unk_piece: unknown token id, 값
* bos_id, bos_piece: begin of sentence token id, 값
* eos_id, eos_piece: end of sentence token id, 값
* user_defined_symbols: 사용자 정의 토큰

----
vocab 생성이 완료되면 imdb.model, imdb.vocab 파일 두개가 생성됨

vocab 파일에서 학습된 서브워드를 확인할 수 있음

In [9]:
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
895,▁found,-892
4019,▁helped,-4016
3048,oph,-3045
1004,ney,-1001
393,fter,-390
2437,▁IMD,-2434
2,</s>,0
1101,not,-1098
1310,▁light,-1307
4115,▁asked,-4112


In [10]:
# model 파일 로드 --> 단어 시퀀스를 정수 시퀀스로 변환(인코딩) 또는 반대(디코딩) 가능
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"

sp.load(vocab_file)

True

In [14]:
# test!!
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]

for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))  # 서브워드 시퀀스로 변환
    print(sp.encode_as_ids(line))  # 정수로 인코딩
    print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]



* GetPieceSize() : 단어집합 크기 확인 가능

In [15]:
sp.GetPieceSize()

5000

* idToPiece : 정수--> 서브워드로 변환

In [16]:
sp.IdToPiece(430)

'▁character'

In [18]:
sp.PieceToId('▁character')

430

* DecodeIds : 정수 시퀀스를 문장으로 변환
* DecodePieces : 서브시퀀스를 문장으로 변환

In [19]:
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91])

'I have waited a long time for someone to film'

In [20]:
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])

'I have waited a long time for someone to film'

* encode : 문장 --> 인자값에 따라 정수시퀀스 또는 서브워드 시퀀스로 변환

In [21]:
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]


### 3. 네이버 영화 리뷰 토큰화하기
-----
네이버 영화리뷰 사례 

In [22]:
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [23]:
# 데이터 다운로드 및 저장
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", \
                           filename="ratings.txt")

naver_df = pd.read_table("ratings.txt")
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [24]:
naver_df.shape  # 20만개 샘플 존재

(200000, 3)

In [25]:
# Null 값 확인 후 제거
naver_df.isnull().sum()

id          0
document    8
label       0
dtype: int64

In [26]:
naver_df = naver_df.dropna(how='any')
naver_df.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [27]:
naver_df.shape  # 최종 샘플

(199992, 3)

In [28]:
# 입력하기 위해 txt 파일로 전환
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [29]:
spm.SentencePieceTrainer.Train(
    "--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe \
    --max_sentence_length=9999")

In [30]:
# 생성된 파일 확인
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,▁그,-6


In [31]:
vocab_list.sample(10)

,0,1
4807,핰,-4804
1633,▁저는,-1630
2244,▁왠지,-2241
4739,낰,-4736
646,▁어디,-643
853,▁황,-850
3409,랑,-3406
1531,▁멋있,-1528
2077,세가,-2074
4634,퍽,-4631


In [32]:
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [33]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]

for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[132, 966, 1296, 2590, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]



In [34]:
sp.GetPieceSize()

5000

In [35]:
# 정수에서 서브워드 맵핑 확인
sp.IdToPiece(4)

'영화'

In [36]:
sp.PieceToId('영화')

4

In [37]:
# 정수시퀀스에서 문장으로 변환
sp.DecodeIds([54, 200, 821, 85])

'진짜 최고의 영화입니다 ᄏᄏ'

In [38]:
sp.DecodePieces(['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ'])

'진짜 최고의 영화입니다 ᄏᄏ'